# Preprocess house price

Get house price per OA across England as a Parquet file

## A new attribute-linked residential property price dataset for England and Wales 2011-2019

We can use https://reshare.ukdataservice.ac.uk/854942/ that derived a price per sqm from link between sales data and EPC.

In [1]:
import pandas as pd

In [2]:
# It is 6.3GB...
linked_epc_path = "https://reshare.ukdataservice.ac.uk/854942/1/tranall2011_19.csv"

Filter only Tyne and Wear for the last two years (2018-19).

In [ ]:
epc = pd.read_csv(linked_epc_path)
epc["dateoftransfer"] = pd.to_datetime(epc.dateoftransfer)
last2years = epc[epc.dateoftransfer > datetime.datetime(2018, 1, 1)]

Link to geometry

In [ ]:
oa = gpd.read_file(   "https://borders.ukdataservice.ac.uk/ukborders/easy_download/prebuilt/shape/infuse_oa_lyr_2011_clipped.zip", engine="pyogrio"
)
agg = (
    last2years[["oa11", "priceper", "numberrooms", "price", "tfarea"]]
    .groupby("oa11")
    .mean()
    .reset_index()
)
agg_oa = oa.merge(agg, left_on="geo_code", right_on="oa11", how="inner")
agg_oa

In [ ]:
agg_oa.to_file(
    f"{data_folder}/processed/house_prices/price_per_sqm.gpkg", engine="pyogrio"
)